In [1]:
# Redact later 
# pixabay API key
key = ""

# Genanki for tamil

This Juputer notebook is a modified version of [jm-moreau](https://github.com/jm-moreau) French Anki card creation notebook. Link to his depository:  (https://github.com/jm-moreau/french-genanki-jupyter)
The notebook uses 1000 most common Tamil words web-scraped on a website. Used googletrans python package for translation and IPA, gTTS python package for the creation of pronunciation and finally pixbay for images. The images may not be relavant.

## Steps
1. Webscrape tamil words (top 500 words, or nouns or any list)\
2. Get english translation along with IPA(pronunciation)\
3. Stroring the data as pandas dataframe
4. Get relavant images with Pixabay API. some images might be silly as most of the words are non-specific. For example: as, that, this
5. Using gTTS, get pronunciation mp3 files.
6. Store the file locations in dataframe columns
7. Creating Genanki model -> note -> Deck and finally saving it.

## Instructions
1. Get Pixabay API key from [Pixabay](https://pixabay.com/api/docs/)
2. You may have you tweak the webscraping code if you are using different webistes

  

In [2]:
# installing packages 
! pip install genanki
! pip install googletrans==4.0.0-rc1
! pip install gTTS
! pip install retry


from IPython.display import clear_output
clear_output()
print("Packages are installed")

Packages are installed


In [3]:
#importing packages 
import random
from pathlib import Path
import yaml
import requests
from textwrap import dedent
import os
import pandas as pd
from bs4 import BeautifulSoup
import genanki
from gtts import gTTS
print("Packages are imported")

Packages are imported


Getting tamil words

In [4]:
# Variables. Please change if neeeded
website_url = "https://1000mostcommonwords.com/1000-most-common-tamil-words/"
order_of_the_table = 0 # if the table of interest is first then put zero or 1 so on

In [5]:
# Fetching the table to datafram
page = requests.get(website_url)
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.find_all('table')[order_of_the_table]
df = pd.read_html(str(table))[0]
# Drop first column of dataframe
df = df.iloc[: , 1:]
# Drop first row
df = df.iloc[1:501 , :]
# Rename the columns
df = df.rename(columns={1: "Tamil", 2: "English"})
df.head()

,Tamil,English
1,என,as
2,நான்,I
3,அவரது,his
4,என்று,that
5,அவர்,he


In [6]:
# defining functions for translation and IPA
from googletrans import Translator
from retry import retry

translator = Translator()

@retry(Exception, delay=5*60, tries=-1)
def get_translation(word):
  ''' word: String (english)
      output: String (tamil tranlation)
  '''
  translation = translator.translate(word, dest='ta')
  return translation.text

@retry(Exception, delay=5*60, tries=-1)
def get_IPA(word):
  ''' word: String (english)
      output: String (tamil pranonciation)
  '''
  translation = translator.translate(word, dest='ta')
  return translation.pronunciation

In [7]:
#Getting IPA
from tqdm import tqdm
df["IPA"] = ""

for index, row in tqdm(df.iterrows()):
  row['IPA'] = get_IPA(row['Tamil'])

500it [00:20, 24.16it/s]


In [8]:
df.tail()

,Tamil,English,IPA
496,ஆட்சி,govern,Āṭci
497,இழுக்க,pull,Iḻukka
498,குளிர்,cold,Kuḷir
499,அறிவிப்பு,notice,Aṟivippu
500,குரல்,voice,Kural


In [ ]:
#Creating functions to retrive url for image search and download. The hourly limit is 500 requests

image_serach_URL = "https://pixabay.com/api/"

def getimageURL(word):
  PARAMS = {'key':key, "q":word}
  r = requests.get(url = image_serach_URL, params = PARAMS)
  data = r.json()
  try:
    return data['hits'][0]['webformatURL']
  except IndexError:
    PARAMS = {'key':key, "q":str(random.randint(0, 99))}
    r = requests.get(url = image_serach_URL, params = PARAMS)
    data = r.json()
    return data['hits'][0]['webformatURL']

def download_image(url, filename): # Please pass word as word.mp3 or word.img
  if Path(filename + ".jpg").is_file():
    full_file_name = filename + str(random.randint(0, 99)) + ".jpg" #if the file exists then create a new one
  else:
    full_file_name = filename + ".jpg"  # create a file name with english word
  
  with open(full_file_name, 'wb') as f:
    file_data = requests.get(url)
    file_data.raise_for_status()
    f.write(file_data.content)
  return full_file_name

In [ ]:
#################################################################### 
# I am lazy to get bing image search API. the Pixabay api has limited requests per hours
#################################################################### 

# #downloading images
# # path = Path.cwd() / 'image_files'
# path = Path('/content/image_files')
# path.mkdir(exist_ok=True)  #Creating subfolder for sound files
# os.chdir(path) # changing the CWD

# #creating empty column
# df['Image_file_location'] = ""
# df['Image_file_location_full_path'] = ""

# for index, row in tqdm(df.iterrows()):
#   img_url = getimageURL(row['English'])
#   file_name = download_image(img_url, row['English'])
#   row['Image_file_location'] = file_name
#   row['Image_file_location_full_path'] = '/content/image_files/' + file_name

In [9]:
df.head()

,Tamil,English,IPA
1,என,as,Eṉa
2,நான்,I,Nāṉ
3,அவரது,his,Avaratu
4,என்று,that,Eṉṟu
5,அவர்,he,Avar


In [10]:
#creating download mp3 function
def download_mp3(word):
  if Path(word + ".mp3").is_file():
    full_file_name = word + str(random.randint(0, 99)) + ".mp3" #if the file exists then create a new one
  else:
    full_file_name = word + ".mp3"  # create a file name with english word
  
  tts = gTTS(word, lang='ta')
  tts.save(full_file_name)
  return full_file_name

In [11]:
#downloading mp3
path = Path('/content/sound_files')
path.mkdir(exist_ok=True)  #Creating subfolder for sound files
os.chdir(path) # changing the CWD

df['sound_file_location'] = ""
df['sound_file_location_full_path'] = ""

for index, row in tqdm(df.iterrows()):
  file_name = download_mp3(row['Tamil'])
  row['sound_file_location'] = file_name
  row['sound_file_location_full_path'] = '/content/sound_files/' + file_name

500it [01:13,  6.83it/s]


In [12]:
df.head()

,Tamil,English,IPA,sound_file_location,sound_file_location_full_path
1,என,as,Eṉa,என.mp3,/content/sound_files/என.mp3
2,நான்,I,Nāṉ,நான்.mp3,/content/sound_files/நான்.mp3
3,அவரது,his,Avaratu,அவரது.mp3,/content/sound_files/அவரது.mp3
4,என்று,that,Eṉṟu,என்று.mp3,/content/sound_files/என்று.mp3
5,அவர்,he,Avar,அவர்.mp3,/content/sound_files/அவர்.mp3


In [13]:
#Creating unique Ids for genanki (should also be saved)
path = Path('/content')
os.chdir(path) # changing the CWD

def create_id():
    return random.randrange(1 << 30, 1 << 31)

genanki_id_yaml = "genanki_ids.yaml"

if not Path(genanki_id_yaml).is_file():
    with open(genanki_id_yaml, 'wt') as f:
        f.write('deck_id: ' + str(create_id()) + '\n') 
        f.write('model_id: ' + str(create_id()) + '\n') 
# load IDs
ids = {}
with open(genanki_id_yaml, 'r') as stream:
    try:
        ids = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [14]:
# Create Anki model
my_model = genanki.Model(
  int(ids['model_id']),
  'Tamil (Genaki)',
  fields=[
    {'name': 'Tamil'},
    {'name': 'English'},
    {'name': 'IPA'},
    {'name': 'Audio'},
    # {'name': 'Picture'},
  ],
  templates=[
    # {
    #   'name': 'Tamil2English',
    #   'qfmt': "<div style='font-family: Arial; font-size: 20px;'>{{Tamil}}</div>",
    #   'afmt': dedent("""\
    #             <div style='font-family: Arial; font-size: 20px;'>{{English}}</div>
    #             <div style='font-family: Arial; font-size: 20px;'>{{Tamil}}</div>
    #             <div style='font-family: Arial; font-size: 20px;'>{{Audio}}</div>
    #             <div style='font-family: Arial; font-size: 20px;'>{{IPA}}</div>
    #             <div style='font-family: Arial; font-size: 20px;'>{{Picture}}</div>
    #             """),
    # },
    {
      'name': 'Tamil2English',
      'qfmt': "<div style='font-family: Arial; font-size: 20px;'>{{Tamil}}</div>",
      'afmt': dedent("""\
                <div style='font-family: Arial; font-size: 20px;'>{{English}}</div>
                <div style='font-family: Arial; font-size: 20px;'>{{Tamil}}</div>
                <div style='font-family: Arial; font-size: 20px;'>{{Audio}}</div>
                <div style='font-family: Arial; font-size: 20px;'>{{IPA}}</div>
                """),
    },

    # {
    #   'name': 'English2Tamil',
    #   'qfmt': "<div style='font-family: Arial; font-size: 20px;'>{{English}}</div>",
    #   'afmt': dedent("""\
    #             <div style='font-family: Arial; font-size: 20px;'>{{English}}</div>
    #             <div style='font-family: Arial; font-size: 20px;'>{{Tamil}}</div>
    #             <div style='font-family: Arial; font-size: 20px;'>{{Audio}}</div>
    #             <div style='font-family: Arial; font-size: 20px;'>{{IPA}}</div>
    #             <div style='font-family: Arial; font-size: 20px;'>{{Picture}}</div>
    #             """),
    # },

    {
      'name': 'English2Tamil',
      'qfmt': "<div style='font-family: Arial; font-size: 20px;'>{{English}}</div>",
      'afmt': dedent("""\
                <div style='font-family: Arial; font-size: 20px;'>{{English}}</div>
                <div style='font-family: Arial; font-size: 20px;'>{{Tamil}}</div>
                <div style='font-family: Arial; font-size: 20px;'>{{Audio}}</div>
                <div style='font-family: Arial; font-size: 20px;'>{{IPA}}</div>
                """),
    },
  ],
css=""".card {
 font-family: arial;
 font-size: 20px;
 text-align: center;
 color: black;
 background-color: white;
}"""
)

In [20]:
# create notes for each result 
my_notes = []
for index, row in tqdm(df.iterrows()):
  my_note = genanki.Note(
      model=my_model,
      fields=[
              row['Tamil'],
              row['English'],
              row['IPA'],
              # '<img src="{}">'.format(row['Image_file_location']),
               "[sound:{}]".format(row['sound_file_location']), #[sound:sound.mp3] format for anki decks
      ])
  my_notes.append(my_note)

500it [00:00, 12418.44it/s]


In [16]:
# create deck and add notes
my_deck = genanki.Deck(
  int(ids['deck_id']),
  'Tamil-genanki')

for note in my_notes:
    my_deck.add_note(note)

In [17]:
# media = df['Image_file_location_full_path'].tolist()
# media.extend(df['sound_file_location_full_path'].tolist())
media = df['sound_file_location_full_path'].tolist()
media[:10]

['/content/sound_files/என.mp3',
 '/content/sound_files/நான்.mp3',
 '/content/sound_files/அவரது.mp3',
 '/content/sound_files/என்று.mp3',
 '/content/sound_files/அவர்.mp3',
 '/content/sound_files/இருந்தது.mp3',
 '/content/sound_files/ஐந்து.mp3',
 '/content/sound_files/ம்.mp3',
 '/content/sound_files/உள்ளன.mp3',
 '/content/sound_files/உடன்.mp3']

In [18]:
my_package = genanki.Package(my_deck)
my_package.media_files = media

In [19]:
my_package.write_to_file('output.apkg')